# Color Detection

In [2]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [3]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [4]:
color = [194, 137, 50] # color in BGR colorspace

first = False
firstFrame = 0
capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()
    frame = cv2.flip(frame, 1)
    
    if (first == False):
        firstFrame = frame
    
    frame_blur = cv2.GaussianBlur(frame, (11, 11), 9) # blurring the image may help get the desired result, but it can be removed
    
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV
    lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
    
    mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    bboxes = list()
    for cnt in contours:
        if cv2.contourArea(cnt) > 100:  # only continues if size of the object is large enough (removes noise)
            x1, y1, w, h = cv2.boundingRect(cnt)  # finds a bounding box for each object
            c = list([int(x1 + w/2), int(y1 + h/2)])  # centerpoint of bbox
            # check other bboxes to see if we want to combine them into 1 box
            newBox = list([c[0], c[1], x1, y1, w, h])
            for i in bboxes[:]:
                cxi, cyi, x1i, y1i, wi, hi = i  # bbox we check the newBox against
                if np.sqrt((c[0] - cxi)**2 + (c[1] - cyi)**2) < np.sqrt(w**2 + h**2)/3 + np.sqrt(wi**2 + hi**2)/3 + 25: # if centerpoints are close enough (scales with box size)
                    bboxes.remove(i)
                    # reassign bbox boundaries so the new box contains both nearby boxes
                    newBox[2], newBox[3] = min(x1, x1i), min(y1, y1i)  # reassigns x1 and y1 values
                    newBox[4], newBox[5] = max(x1+w, x1i+wi) - newBox[2], max(y1+h, y1i+hi) - newBox[3]  # reassgins w and h values
                    newBox[0], newBox[1] = int(newBox[2] + newBox[4]/2), int(newBox[3] + newBox[5]/2)  # reassigns centerpoint values
            bboxes.append(newBox)
                
    for i in bboxes:  # draws bboxes after we have curated the list
        cx, cy, x1, y1, w, h = i
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)
    
    cv2.imshow('blur', frame_blur)
    cv2.imshow('mask', mask)
    cv2.imshow('webcam', frame)
    if cv2.waitKey(40) & 0xFF == ord(' '):
        break  # ends loop when spacebar is pressed

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when spacebar is pressed

151, 210, 151
159, 217, 159
161, 219, 161
163, 221, 163
165, 223, 165
168, 225, 168
178, 234, 178
179, 235, 179
180, 236, 180
180, 237, 180
181, 237, 181
182, 238, 182
181, 238, 181
182, 238, 182
181, 237, 181
181, 237, 181
181, 237, 181
182, 236, 182
181, 236, 181
181, 235, 181
181, 234, 181
182, 236, 182
183, 238, 183
183, 238, 183
183, 238, 183
184, 238, 184
184, 238, 184
184, 237, 184
184, 237, 184
184, 235, 184
183, 234, 183
182, 232, 182
181, 231, 181
179, 231, 179
176, 229, 176
176, 227, 176
173, 225, 173
173, 224, 173
172, 224, 172
172, 224, 172
172, 223, 172
172, 223, 172
172, 223, 172
170, 222, 170
169, 221, 169
169, 221, 169
169, 221, 169
169, 221, 169
169, 222, 169
170, 222, 170
170, 222, 170
171, 224, 171
171, 223, 171
171, 223, 171
171, 224, 171
171, 224, 171
172, 224, 172
172, 224, 172
172, 172, 314
172, 224, 172
171, 224, 171
171, 224, 171
170, 223, 170
169, 223, 169
170, 220, 170
297, 322, 297
171, 171, 297
171, 221, 171
170, 222, 170
171, 222, 171
171, 222, 171
172, 2